### Statistical Arbitrage using Graph Theory

Implmentation of "Statistical arbitrage in multi-pair trading strategy based on graph clustering algorithms in US equities market" by Adam Korniejczuka & Robert Slepaczukb.


#### Methodology:

1. Data Collection
- Universe: S&P 500 constituents (updated historically as they change).
- Timeframe: Daily data from 2000 to 2022.
- Price Data: Adjusted close prices for all stocks.

2. Graph Based clustering of the stocks
- Calculate residual returns (on Fama-French 3 factor model)
- Compute correlation matrix of residuals over a rolling window (30 days in paper)
- Treat the correlation matrix as an adjacency matrix of a signed, weighted, undirected graph.
- Use SPONGE-sym clustering algorithm to identify clusters of tightly connected stocks.

3. Signal Generation
- Every k days (e.g., every 10 days) for each cluster:
- Compute cluster mean return over past 5 days.
- Signal is then Stock’s 5-day return - Cluster’s 5-day mean return
- If signal is significantly:
    - Below cluster average then Long
    - Above cluster average then Short

4. Feature Engineering for Signal Quality
- For each signal generated extract features like:
- Graph-based:
    - Local/global vertex degree
    - Cluster size
    - Graph density
    - Cluster size
- Price-based:
    - Signal value
    - Sign of deviation
    - Stock & cluster average returns over past 10 days

5. Label Signals
- Label based on profitability (could try Triple Barrier Method as well), paper used "If the stock’s return after the signal exceeds a threshold (e.g., 4%) or beats transaction cost."
- This becomes a target for ML Classification

6. Machine Learning Signal Classifier
- Train multiple classifiers on the labeled dataset:
    - Logistic Regression
    - Gradient Boosted Trees
    - Neural Net (MLP)
    - SGD Classifier
    - AdaBoost
- Ensemble (soft voting) using classifier probabilities.

7. Trade Filtering 
- Only trade signals where ensemble confidence > 0.6 to reduce bad trades and minimize transaction costs.

8. Bet Sizing
- Use Kelly Critereon

9. Apply time-decaying take profit and stop loss:
- Threshold shrinks each day since entry.
- Threshold also weighted by signal confidence.

10. Performance Evaluation
- Backtest using realistic assumptions:
    - Transaction cost = 0.05%
    - Fractional shares allowed
- Metrics:
    - Annualized Return
    - Sharpe & Sortino Ratio
    - Max Drawdown
    - Calmar Ratio
    - Modified Information Ratios (IR*, IR**)


### Step 1: Data Collection

In [1]:
# Smaller Dataset (to start)
import pandas as pd
import numpy as np
import yfinance as yf
from datetime import datetime, timedelta

sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")[0]
tickers = sp500['Symbol'].tolist()

# Remove RF (confuses with risk free rate later)
tickers = [ticker for ticker in tickers if ticker != 'RF']

#Change any . to -
tickers = [ticker.replace('.', '-') for ticker in tickers]

start_date = "2014-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

df_data = yf.download(
    tickers,
    start=start_date,
    end=end_date,
    progress=True,
    group_by='ticker',
    auto_adjust=True,
    threads=True
)

# Extract just Close prices into a single DataFrame
df_prices = pd.concat(
    [df_data[ticker]['Close'].rename(ticker) for ticker in tickers if ticker in df_data],
    axis=1
)

threshold = int(0.95 * len(df_prices))  # Allow up to 5% NaNs
df_prices = df_prices.dropna(axis=1, thresh=threshold)



[*********************100%***********************]  501 of 501 completed


In [2]:
# Price to Returns
def compute_log_returns(price_df: pd.DataFrame) -> pd.DataFrame:
    """
    Compute returns from a price DataFrame.
    """
    # Ensure index is datetime
    price_df.index = pd.to_datetime(price_df.index)
    
    # Calculate log returns
    returns_df = price_df / price_df.shift(1)
    
    # Drop rows with NaNs introduced by shift
    returns_df = np.log(price_df / price_df.shift(1)).dropna()
    
    return returns_df

# Compute returns

df_returns = compute_log_returns(df_prices)

In [3]:
# Load Fama-French daily 3-factor data
ff_factors = pd.read_csv("F-F_Research_Data_Factors_daily.CSV", skiprows=3, index_col=0)

# Use first row as header
ff_factors = ff_factors.drop(ff_factors.index[0])

ff_factors.dropna(inplace=True)
ff_factors.columns = ['MKT', 'SMB', 'HML', 'RF']
# Ensure all factor columns are float
ff_factors[["MKT", "SMB", "HML", "RF"]] = ff_factors[["MKT", "SMB", "HML", "RF"]].astype(float)
ff_factors.index = pd.to_datetime(ff_factors.index, format="%Y%m%d")

# Align returns and factors
df_returns_ff = df_returns.join(ff_factors[["MKT", "SMB", "HML", "RF"]], how="inner")

# Subtract RF from stock returns (excess returns only on stock columns)
stock_cols = df_returns.columns
excess_returns = df_returns_ff[stock_cols].sub(df_returns_ff["RF"], axis=0)
factors = df_returns_ff[["MKT", "SMB", "HML"]]

In [4]:
from sklearn.linear_model import LinearRegression

def compute_residuals(excess_returns, factors):
    residuals = pd.DataFrame(index=excess_returns.index, columns=excess_returns.columns)
    model = LinearRegression()

    for ticker in excess_returns.columns:
        y = excess_returns[ticker].dropna()
        X = factors.loc[y.index]
        model.fit(X, y)
        if len(y) < 30:
            continue  # skip short series
        y_pred = model.predict(X)
        residuals.loc[y.index, ticker] = y - y_pred

    return residuals.astype(float)

residuals = compute_residuals(excess_returns, factors)

### Step 2: Graph Based Clustering

In [5]:
# Use a dictionary, but could later convert to tensor if needed
def rolling_signed_corr(residuals: pd.DataFrame, window: int = 60) -> dict:
    """
    Compute rolling signed correlation matrices from residual returns.

    Returns: 
    corr_matrices : dict
        Dictionary mapping end-of-window date to signed correlation matrix.
    """
    corr_matrices = {}
    for end in range(window, len(residuals)):
        window_data = residuals.iloc[end - window:end]
        date_key = residuals.index[end]
        corr = window_data.corr()
        corr_matrices[date_key] = corr
    return corr_matrices

# Compute rolling signed correlation matrices
rolling_corrs = rolling_signed_corr(residuals, window=30)

In [6]:
# Now cluster using SPONGEsym Algorithm

# %pip install git+https://github.com/alan-turing-institute/SigNet.git
from signet.cluster import Cluster
from scipy.sparse import csc_matrix

def estimate_k_variance(corr_matrix, threshold=0.90):
    eigenvalues = np.linalg.eigvalsh(corr_matrix.fillna(0).values)
    eigenvalues = np.flip(np.sort(eigenvalues))  # descending order
    cumulative_variance = np.cumsum(eigenvalues) / np.sum(eigenvalues)
    k = np.searchsorted(cumulative_variance, threshold) + 1
    return k

def run_sponge_sym(corr_matrix: pd.DataFrame, k: int = 6) -> dict:
    """
    Applies SPONGE-sym clustering to a signed correlation matrix using SigNet.

    Parameters:
    -----------
    corr_matrix : pd.DataFrame
        Signed correlation matrix (symmetric with values in [-1, 1]).
    k : int
        Number of clusters to generate.

    Returns:
    --------
    cluster_map : dict
        Dictionary mapping tickers to cluster labels.
    """
    W = corr_matrix.fillna(0).values
    Ap = np.where(W > 0, W, 0)
    An = np.where(W < 0, -W, 0)

    Ap_sparse = csc_matrix(Ap)
    An_sparse = csc_matrix(An)

    c = Cluster((Ap_sparse, An_sparse))
    labels = c.SPONGE_sym(k=k, tau_p=1.0, tau_n=1.0)  # ← correct function name

    return dict(zip(corr_matrix.index, labels))

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="signet.cluster")

sponge_cluster_results = {}

for date, corr_matrix in rolling_corrs.items():
    try:
        k_est = estimate_k_variance(corr_matrix, threshold=0.90)
        clusters = run_sponge_sym(corr_matrix, k=k_est)
        sponge_cluster_results[date] = clusters
    except Exception as e:
        print(f"SPONGE clustering failed on {date}: {e}")

### Step 3: Signal Generation

Every k days (e.g., every 10 days) for each cluster:
- Compute cluster mean return over past 5 days.
- Signal is then Stock’s 5-day return - Cluster’s 5-day mean return
- If signal is significantly:
    - Below cluster average then Long
    - Above cluster average then Short

In [7]:
# Parameters
signal_spacing = 10
lookback_days = 10

# If a stock outperformed its cluster its signal is positive
# If a stock underperformed its cluster its signal is negative

# Generate signal every `signal_spacing` days
signal_dates = sorted(sponge_cluster_results.keys())[::signal_spacing]
signals = []

for date in signal_dates:
    if date not in df_returns.index:
        continue

    try:
        date_idx = df_returns.index.get_loc(date)
    except KeyError:
        continue

    window_start = date_idx - lookback_days
    if window_start < 0:
        continue  # skip if not enough data

    past_window = df_returns.iloc[window_start:date_idx]
    returns_past = past_window.sum()

    # Cluster assignments from SPONGE
    clusters = sponge_cluster_results[date]
    cluster_series = pd.Series(clusters)

    for cluster_id in set(cluster_series.values):
        members = cluster_series[cluster_series == cluster_id].index

        if len(members) < 2:
            continue

        cluster_mean = returns_past[members].mean()

        for stock in members:
            signal_value = returns_past[stock] - cluster_mean

            signals.append({
                "date": date,
                "stock": stock,
                "cluster": cluster_id,
                "signal": signal_value,
                "cluster_mean": cluster_mean,
                "stock_return": returns_past[stock]
            })

# Create signal DataFrame
signals_df = pd.DataFrame(signals)
signals_df.sort_values(["date", "signal"], ascending=[True, False], inplace=True)
signals_df.reset_index(drop=True, inplace=True)

# Quantile-based signal thresholds
lower, upper = signals_df["signal"].quantile([0.10, 0.90])

# Assign positions
signals_df["position"] = 0
signals_df.loc[signals_df["signal"] <= lower, "position"] = 1    # Long
signals_df.loc[signals_df["signal"] >= upper, "position"] = -1   # Short

In [8]:
signals_df.head(5)

,date,stock,cluster,signal,cluster_mean,stock_return,position
0,2014-09-15,PANW,3,0.169870,0.025216,0.195085,-1
1,2014-09-15,ENPH,12,0.157938,0.025218,0.183156,-1
2,2014-09-15,AXON,7,0.137672,0.024560,0.162232,-1
3,2014-09-15,ULTA,12,0.134709,0.025218,0.159926,-1
4,2014-09-15,AVGO,5,0.117919,0.022253,0.140172,-1


### 4. Feature Engineering for Signal Quality
- For each signal generated extract features:
- Graph-based:
    - Local/global vertex degree
    - Cluster size
    - Graph density
    - Cluster size
- Price-based:
    - Signal value
    - Sign of deviation
    - Stock & cluster average returns over past 10 days

In [9]:
# Adding features to train ML on

# Ensure returns are sorted by date
df_returns = df_returns.sort_index()

# Enrich signals with features
feature_rows = []

for idx, row in signals_df.iterrows():
    date = row['date']
    stock = row['stock']
    cluster_id = row['cluster']

    # Retrieve cluster members
    cluster_members = [s for s, c in sponge_cluster_results[date].items() if c == cluster_id]
    if len(cluster_members) < 2 or stock not in cluster_members:
        continue

    # Retrieve correlation matrix for that date
    corr_matrix = rolling_corrs.get(date)
    if corr_matrix is None or stock not in corr_matrix.index:
        continue

    # 1. Local Degree: sum of abs correlations to others
    local_degree = corr_matrix.loc[stock].drop(stock).abs().sum()

    # 2. Cluster Density: average abs correlation between all cluster members
    sub_corr = corr_matrix.loc[cluster_members, cluster_members]
    tri_mask = np.triu(np.ones(sub_corr.shape), k=1).astype(bool)
    flat_vals = sub_corr.values[tri_mask]
    try:
        density = np.nanmean(np.abs(flat_vals))  # alt: density = (2 * np.count_nonzero(flat_vals)) / (n * (n - 1))
    except:
        continue

    # 3. Cluster Size
    clust_size = len(cluster_members)

   # 4. Global (normalised) vertex degree: mean |ρ| of the stock to every other stock in the graph
    global_degree = corr_matrix.loc[stock].drop(stock).abs().mean()

    # 5. “Number-of-clusters ÷ graph size” – graph-level normalised feature
    clusters_per_graph = len(set(sponge_cluster_results[date].values())) / corr_matrix.shape[0]

    # 6. Price-Based Features: stock & cluster return over past 10 days
    try:
        date_idx = df_returns.index.get_loc(date)
        window_start = date_idx - 10
        if window_start < 0:
            continue

        past_window = df_returns.iloc[window_start:date_idx]
        stock_10d_return = past_window[stock].sum()
        cluster_10d_return = past_window[cluster_members].mean(axis=1).sum()
    except Exception as e:
        print(f"Feature extraction failed for {date} - {stock}: {e}")
        continue

    # Append enriched signal
    feature_rows.append({
        **row,
        "local_degree": local_degree,
        "cluster_density": density,
        "cluster_size": clust_size,
        "global_degree": global_degree,
        "clusters_per_graph": clusters_per_graph,
        "signal_sign": np.sign(row["signal"]),
        "abs_signal": np.abs(row["signal"]),
        "stock_10d_return": stock_10d_return,
        "cluster_10d_return": cluster_10d_return
    })


# Build final DataFrame
features_df = pd.DataFrame(feature_rows)

In [10]:
features_df.head()

,date,stock,cluster,signal,cluster_mean,stock_return,position,local_degree,cluster_density,cluster_size,global_degree,clusters_per_graph,signal_sign,abs_signal,stock_10d_return,cluster_10d_return
0,2014-09-15,PANW,3,0.169870,0.025216,0.195085,-1,79.898886,0.368781,22,0.172568,0.049569,1.0,0.169870,0.195085,0.025216
1,2014-09-15,ENPH,12,0.157938,0.025218,0.183156,-1,76.557791,0.301311,31,0.165352,0.049569,1.0,0.157938,0.183156,0.025218
2,2014-09-15,AXON,7,0.137672,0.024560,0.162232,-1,69.245678,0.293857,20,0.149559,0.049569,1.0,0.137672,0.162232,0.024560
3,2014-09-15,ULTA,12,0.134709,0.025218,0.159926,-1,86.476733,0.301311,31,0.186775,0.049569,1.0,0.134709,0.159926,0.025218
4,2014-09-15,AVGO,5,0.117919,0.022253,0.140172,-1,57.730429,0.225714,17,0.124688,0.049569,1.0,0.117919,0.140172,0.022253


### Step 5: Label Signals
- Label based on profitability 
- This becomes a target for ML Classification

In [30]:
#Idea 1 - Label the signals based on future return threshold

labelled_rows = []

profit_threshold = 0.045  # % threshold over 5-day holding (4% + .5% for fees)
holding_period = 10       # in days

for _, row in features_df.iterrows():
    date = row['date']
    stock = row['stock']
    position = row['position']  # -1 for short, 1 for long, 0 for neutral

    try:
        date_idx = df_returns.index.get_loc(date)
        forward_window = df_returns.iloc[date_idx + 1: date_idx + 1 + holding_period]

        # Cumulative log return → convert to standard return
        log_ret_sum = forward_window[stock].sum()
        future_return = np.exp(log_ret_sum) - 1

        pnl = future_return * position
        label = 1 if (pnl >= profit_threshold) or (pnl >= 0.005) else 0

        labelled_rows.append({
            **row,
            "log_ret_sum": log_ret_sum,
            "future_return": future_return,
            "pnl": pnl,
            "label": label
        })

    except (KeyError, IndexError):
        continue  # not enough forward data, skip

labelled_df = pd.DataFrame(labelled_rows)


In [33]:
# What is the ratio of the signals (0 to 1)?

signal_ratio = labelled_df['label'].value_counts(normalize=True)
print("Signal Ratio (0 to 1):")
print(signal_ratio)

Signal Ratio (0 to 1):
label
0    0.90745
1    0.09255
Name: proportion, dtype: float64


### Step 6: Machine Learning Signal Classifier
Train multiple classifiers on the labeled dataset:

- Logistic Regression
- Gradient Boosted Trees
- Neural Net (MLP)
- SGD Classifier
- AdaBoost

Ensemble (soft voting) using classifier probabilities.

In [34]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Features to use
feature_cols = [
    'signal', 'abs_signal', 'signal_sign',
    'local_degree', 'cluster_density', 'cluster_size',
    'stock_10d_return', 'cluster_10d_return', 'global_degree', 'clusters_per_graph'
]

X = labelled_df[feature_cols]
y = labelled_df['label']

labelled_df = labelled_df.sort_values('date')          # 'date' is a column in the DF

# Use the first 150 unique dates as cutoff (10 day holding period) = 1500 days

cutoff_date = np.sort(labelled_df['date'].unique())[150]   

train_mask = labelled_df['date'] <= cutoff_date
test_mask  = ~train_mask

X_train = labelled_df.loc[train_mask, feature_cols]
y_train = labelled_df.loc[train_mask, 'label']
X_test  = labelled_df.loc[test_mask,  feature_cols]
y_test  = labelled_df.loc[test_mask,  'label']

scaler   = StandardScaler()
X_train  = scaler.fit_transform(X_train)
X_test   = scaler.transform(X_test)

In [37]:
from sklearn.ensemble import AdaBoostClassifier, VotingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

#Hyperparameters from grid search in paper

logreg_best = LogisticRegression(
    C=8,
    penalty='l2',
    solver='lbfgs',
    max_iter=75,
    class_weight='balanced',
    warm_start=False,
    random_state=42
)


SGD_best = SGDClassifier(
    loss='modified_huber',
    penalty='l2',
    alpha=0.001,
    max_iter=200,
    early_stopping=False,
    learning_rate='optimal',
    eta0=0.01,  # required even if not strictly used in 'constant' mode
    warm_start=False,
    random_state=42
)


HGB_best = HistGradientBoostingClassifier(
    learning_rate=0.1,
    early_stopping='auto',
    max_iter=100,
    warm_start=False,
    random_state=42
)


MLP_best = make_pipeline( 
    StandardScaler(),
    MLPClassifier(
        hidden_layer_sizes=(64, 64),
        activation='relu',
        alpha=0.000001,
        learning_rate='constant',
        batch_size=200,
        solver='adam',
        max_iter=500,
        random_state=42
    )
)

ADA_best = AdaBoostClassifier(
    n_estimators=100,
    learning_rate=0.001,
    random_state=42
)

# Define models
models = {
    "LogReg": logreg_best,
    "HGB": HGB_best,
    "MLP": MLP_best,
    "SGD": SGD_best,
    "AdaBoost": ADA_best
}

# Fit all base models
for name, model in models.items():
    print(f"Training {name}...")
    model.fit(X_train, y_train)


Training LogReg...
Training HGB...
Training MLP...
Training SGD...
Training AdaBoost...


In [36]:
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, brier_score_loss, precision_score
import pandas as pd

results = {}

results = {}
summary_rows = []  # For final summary table

for name, model in models.items():
    print(f"\nEvaluating {name}...")
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    try:
        # For binary classification, get prob for class 1
        y_proba = model.predict_proba(X_test)[:, 1]
        auc = roc_auc_score(y_test, y_proba)
        brier = brier_score_loss(y_test, y_proba)
    except Exception as e:
        print(f"AUC/Brier error for {name}: {e}")
        auc = None
        brier = None
        y_proba = None

    precision = precision_score(y_test, y_pred, pos_label=1, zero_division=0)
    report = classification_report(y_test, y_pred, output_dict=True, zero_division=0)

    results[name] = {
        "AUC": auc,
        "Brier Score": brier,
        "Precision": precision,
        "Report": report
    }

    # For the final summary table
    summary_rows.append({
        "Classifier": name,
        "Brier Score": round(brier, 3) if brier is not None else None,
        "Precision": round(precision, 3)
    })

# Create summary DataFrame
results_df = pd.DataFrame(summary_rows).sort_values(by="Brier Score", ascending=True)
from IPython.display import display
display(results_df)



Evaluating LogReg...

Evaluating HGB...

Evaluating MLP...

Evaluating SGD...

Evaluating AdaBoost...


,Classifier,Brier Score,Precision
1,HGB,0.056,0.518
2,MLP,0.069,0.487
3,SGD,0.072,0.512
4,AdaBoost,0.084,0.000
0,LogReg,0.107,0.459


In [38]:
# Ensemble

voting_clf = VotingClassifier(
    estimators=[(name, model) for name, model in models.items()],
    voting='soft'
)
voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)
print("Classification Report (Soft Voting Ensemble):\n")
print(classification_report(y_test, y_pred, zero_division=0))

Classification Report (Soft Voting Ensemble):

              precision    recall  f1-score   support

           0       0.92      0.97      0.94     48666
           1       0.51      0.28      0.36      5617

    accuracy                           0.90     54283
   macro avg       0.71      0.62      0.65     54283
weighted avg       0.88      0.90      0.88     54283

